# 2.3A Izbira modela za strojno učenje

V koraku izbira modela za strojno učenje:
- uvozimo podatke pridobljene v koraku 2. s katerih bomo učili modele
- podatke o strukturah molekule pretvorimo v fingerprinte (bitni zapis strukture, s tem dobimo featurje na X osi)
- izberemo kombinacijo najbolj primernega fingerprinta, klasifikatorja, vzorčenjske tehnike, skalarja, tehnike za zaznavanje outlierjev

# Uvoz knjižnic in splošnih funkcij

In [1]:
%run __A_knjiznice.py

from __A_knjiznice import *
from __B_funkcije import *
import __C_konstante as kon
%matplotlib inline

/Users/samonose/PycharmProjects/IDO_inhibitors_ML_predictions/dir1
/Users/samonose/PycharmProjects/IDO_inhibitors_ML_predictions/dir1


# Uvoz obdelanih podatkov obdelanih v koraku 2. Obdelava in analiza podatkov

## Pregled podatkov

We are limited to max 4142 samples. The sample size is quite small but we are limited to exisitng data. 

Two rules of thumb are often considered when we estimate the size of training set.

1. Rule-of-Thumb for Prediction Classes
The first rule suggests having a sample size at least 50 to 1000 times the number of prediction classes. Since we're dealing with binary classification (2 classes), this guideline would technically require a minimum of 100 to 2000 samples. With 4250 samples, we comfortably exceed the lower end of this range, suggesting that, from the perspective of prediction classes alone, our sample size is adequate.
2. Rule-of-Thumb for Observations vs. Features
The more challenging guideline in our case is the one suggesting having at least 20 times the number of observations as features. With up to 4860 features in case of certain fingerprints, this rule would imply you need around 100.000 observations, a number far exceeding our current sample size. This guideline is particularly important in machine learning to avoid overfitting, where a model learns the noise in the training data instead of the actual signal, leading to poor generalization to new data.

In order to be as close as possible to the second rule, I decidet to exclude fingerprints which have more than 1024 features, because fingerprints with +1024 features could lead to overfiting



In [2]:
df = pd.read_csv(f'{kon.path_files}/dp.csv')
df

,Smiles,ROMol,Activity
0,O=C1c2cc([N+](=O)[O-])ccc2-n2c1nc1ccccc1c2=O,<rdkit.Chem.rdchem.Mol object at 0x16fce49e0>,1
1,Cc1cc(C2CC2)ncc1-c1ccc(C2(C(=O)Nc3ccc(F)cc3)CO...,<rdkit.Chem.rdchem.Mol object at 0x16fcb09e0>,1
2,O=C(Nc1ccc(C2(C(=O)Nc3ccc(F)cc3)COC2)cc1)c1ccc...,<rdkit.Chem.rdchem.Mol object at 0x16fcdcba0>,1
3,O=C(Nc1ccc(F)cc1)C1(C2CCC3C(CCCN3c3ccnc(C(F)(F...,<rdkit.Chem.rdchem.Mol object at 0x16fcddcb0>,1
4,O=C1CC(c2c[nH]c3ccc(F)cc23)C(=O)N1,<rdkit.Chem.rdchem.Mol object at 0x16fce25e0>,1
...,...,...,...
4137,FC(F)(F)c1ccc(-c2c[nH]nn2)cc1,<rdkit.Chem.rdchem.Mol object at 0x16fcbef10>,0
4138,c1ccc2[nH]nnc2c1,<rdkit.Chem.rdchem.Mol object at 0x16fcf54d0>,0
4139,Cc1cccc(NC(=O)C(F)(F)F)c1-c1c[nH]nn1,<rdkit.Chem.rdchem.Mol object at 0x16fcb5230>,0
4140,Cc1ccc(N)cc1-c1c[nH]nn1,<rdkit.Chem.rdchem.Mol object at 0x16fcb2500>,0


In [3]:
activity_counts = df['Activity'].value_counts()
print(activity_counts)

Activity
1    2103
0    2039
Name: count, dtype: int64


# 3.3 Izračun relevantnih metrik za model strojnega učenja

## Test different fingerprints

**standard**: Considers paths of a given length. These are hashed fingerprints, with a default length of 1024.
**extended**: Similar to the standard type, but takes rings and atomic properties into account into account.
**graph**: Similar to the standard type by simply considers connectivity.
**hybridization**: Similar to the standard type, but only consider hybridization state.
**estate**: 79 bit fingerprints corresponding to the E-State atom types described by Hall and Kier.
**cdk-atompairs**: CDK's implementation of the atompairs fingerprint
**cdk-substructure**: CDK substructure fingerprint, basically identical to openbabel's fp4.
**pubchem**: 881 bit fingerprints defined by PubChem.
**klekota-roth**: 4860 bit fingerprint defined by Klekota and Roth.
**shortestpath**: A fingerprint based on the shortest paths between pairs of atoms and takes into account ring systems, charges etc.
**rdk-descriptor**: Various molecular descriptors implemented and calculated by RDKit.
**circular**: An implementation of the ECFP6 fingerprint.
**lingo**: An implementation of the LINGO fingerprint.
**rdkit**: Another implementation of a Daylight-like fingerprint by RDKit.
**maccs**: The popular 166 bit MACCS keys described by MDL.
**avalon**: Substructure or similarity Avalon fingerprint.
**atom-pair**: RDKit Atom-Pair fingerprint.
**topological-torsion**: RDKit Topological-Torsion Fingerprint.
**morgan**: RDKit Morgan fingerprint.
**fp2**: OpenBabel FP2 fingerprint, which indexes small molecule fragments based on linear segments of up to 7 atoms in length.
**fp3**: OpenBabel FP3 fingerprint, which is a fingerprint method created from a set of SMARTS patterns defining functional groups.
**fp4**: OpenBabel FP4 fingerprint, which is a fingerprint method created from a set of SMARTS patterns defining functional groups.
**spectrophore** Openbabel implementation of the spectrophore fingerprint (https://github.com/silicos-it/spectrophore).
**mol2vec**: Unsupervised machine learning approach for mulecule representation.

# Pregled kombinacij izbranih fingerprintov in klasifikacijskih modelov

## V tem koraku bi rad preveril katera kombinacija fingerprinta in klasifikacijskega modela je najboljša preden začnem manipulirati s podatki. Ta korak je pomemben zato da v nadaljnih korakih vidim kaj izboljšujem

In [4]:
# https://medium.com/artificialis/why-how-we-split-train-valid-and-test-fb4d6746ede

In [5]:
input_directory = f'{kon.path_files}/molekulski_prstni_odtisi'

generated_fingerprints = [
    'df_rdkit.csv', 'df_circular.csv',
    'df_morgan.csv'
]

In [6]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTENC
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_selection import SelectKBest, chi2, VarianceThreshold
from sklearn.cluster import FeatureAgglomeration
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA  # Import PCA
import numpy as np

# Define classifiers with default parameters
classifiers = {
    "RandomForestClassifier": RandomForestClassifier(n_jobs=-1, random_state=42)
}

# Dimensionality Reduction Methods with default parameters
dim_reduction_methods = {
    "None": None,
    "SelectKBest": SelectKBest(score_func=chi2, k=150),
    "LDA": LinearDiscriminantAnalysis(),
    "FeatureAgglomeration": FeatureAgglomeration(),
    "PCA": PCA()  # Add PCA to the methods
}

# Methods for Handling Imbalanced Data with default parameters
sampling_techniques = {
    "None": None,
    "SMOTENC": SMOTENC(categorical_features=[0, 1], random_state=42),
    "RandomUnderSampler": RandomUnderSampler(random_state=42)
}

# Store results
results_list = []

# Define Stratified k-fold cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# List of specific filenames to process
for filename in generated_fingerprints:  # Assuming generated_fingerprints is defined
    file_path = os.path.join(input_directory, filename)
    
    if os.path.exists(file_path):  # Check if the file exists
        print(f'Processing fingerprint DataFrame: {filename}')
        
        df = pd.read_csv(file_path)
        y = df[['Activity']].values.ravel()  # Assuming 'Activity' is the target
        X = df.iloc[:, 3:]  # Assuming features start from the 4th column

        # Remove constant features
        selector = VarianceThreshold()
        X = pd.DataFrame(selector.fit_transform(X), columns=selector.get_feature_names_out())


        # Split the data into a training set and a hold-out validation set
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

        # Train and evaluate each classifier
        for clf_name, clf in classifiers.items():
            for dr_name, dr_method in dim_reduction_methods.items():
                for fs_name, fs_method in sampling_techniques.items():
                    steps = []
                    if fs_method is not None:
                        steps.append(('feature_selection', fs_method))
                    if dr_method is not None:
                        steps.append(('dim_reduction', dr_method))
                    steps.append(('classifier', clf))
                    
                    # Create the pipeline
                    pipeline = ImbPipeline(steps)

                    # Perform cross-validation
                    cv_results = []
                    for train_index, val_index in cv.split(X_train, y_train):
                        X_train_cv, X_val_cv = X_train.iloc[train_index], X_train.iloc[val_index]
                        y_train_cv, y_val_cv = y_train[train_index], y_train[val_index]

                        # Fit the model
                        pipeline.fit(X_train_cv, y_train_cv)

                        # Evaluate on the validation set
                        y_val_pred = pipeline.predict(X_val_cv)
                        val_accuracy = accuracy_score(y_val_cv, y_val_pred)
                        val_f1 = f1_score(y_val_cv, y_val_pred)
                        val_precision = precision_score(y_val_cv, y_val_pred)
                        val_recall = recall_score(y_val_cv, y_val_pred)
                        val_roc_auc = roc_auc_score(y_val_cv, y_val_pred)

                        # Store the results for this fold
                        cv_results.append({
                            'Val_Accuracy': val_accuracy,
                            'Val_F1': val_f1,
                            'Val_Precision': val_precision,
                            'Val_Recall': val_recall,
                            'Val_ROC_AUC': val_roc_auc,
                        })

                    # Calculate mean metrics across all folds
                    mean_cv_results = pd.DataFrame(cv_results).mean()

                    # Fit the model on the entire training set
                    pipeline.fit(X_train, y_train)

                    # Evaluate on the hold-out validation set
                    y_val_final_pred = pipeline.predict(X_val)
                    val_accuracy_final = accuracy_score(y_val, y_val_final_pred)
                    val_f1_final = f1_score(y_val, y_val_final_pred)
                    val_precision_final = precision_score(y_val, y_val_final_pred)
                    val_recall_final = recall_score(y_val, y_val_final_pred)
                    val_roc_auc_final = roc_auc_score(y_val, y_val_final_pred)

                    # Append results to the list
                    results_temp = {
                        'Fingerprint': filename,  # Use the filename for identification
                        'Feature_Selection': fs_name,
                        'Dim_Reduction': dr_name,
                        'Classifier': clf_name,
                        'CV_Mean_Accuracy': mean_cv_results['Val_Accuracy'],
                        'CV_Mean_F1': mean_cv_results['Val_F1'],
                        'CV_Mean_Precision': mean_cv_results['Val_Precision'],
                        'CV_Mean_Recall': mean_cv_results['Val_Recall'],
                        'CV_Mean_ROC_AUC': mean_cv_results['Val_ROC_AUC'],
                        'Val_Accuracy': val_accuracy_final,
                        'Val_F1': val_f1_final,
                        'Val_Precision': val_precision_final,
                        'Val_Recall': val_recall_final,
                        'Val_ROC_AUC': val_roc_auc_final,
                    }
                    results_list.append(results_temp)
                    print("\nResults:")
                    print(results_temp)

# Create DataFrame from the results list
results_df = pd.DataFrame(results_list)

Processing fingerprint DataFrame: df_rdkit.csv

Results:
{'Fingerprint': 'df_rdkit.csv', 'Feature_Selection': 'None', 'Dim_Reduction': 'None', 'Classifier': 'RandomForestClassifier', 'CV_Mean_Accuracy': 0.874734909069368, 'CV_Mean_F1': 0.8776613376353335, 'CV_Mean_Precision': 0.8699877128232869, 'CV_Mean_Recall': 0.8858449908153172, 'CV_Mean_ROC_AUC': 0.8745644908674061, 'Val_Accuracy': 0.8745476477683957, 'Val_F1': 0.8743961352657005, 'Val_Precision': 0.8894348894348895, 'Val_Recall': 0.8598574821852731, 'Val_ROC_AUC': 0.8747816822691071}

Results:
{'Fingerprint': 'df_rdkit.csv', 'Feature_Selection': 'SMOTENC', 'Dim_Reduction': 'None', 'Classifier': 'RandomForestClassifier', 'CV_Mean_Accuracy': 0.8699060846741672, 'CV_Mean_F1': 0.8719278979960394, 'CV_Mean_Precision': 0.8709175199471426, 'CV_Mean_Recall': 0.873362653666808, 'CV_Mean_ROC_AUC': 0.8698551884870316, 'Val_Accuracy': 0.8769601930036188, 'Val_F1': 0.8771084337349397, 'Val_Precision': 0.8899755501222494, 'Val_Recall': 0.86460

In [7]:
results_df

,Fingerprint,Feature_Selection,Dim_Reduction,Classifier,CV_Mean_Accuracy,CV_Mean_F1,CV_Mean_Precision,CV_Mean_Recall,CV_Mean_ROC_AUC,Val_Accuracy,Val_F1,Val_Precision,Val_Recall,Val_ROC_AUC
0,df_rdkit.csv,None,None,RandomForestClassifier,0.874735,0.877661,0.869988,0.885845,0.874564,0.874548,0.874396,0.889435,0.859857,0.874782
1,df_rdkit.csv,SMOTENC,None,RandomForestClassifier,0.869906,0.871928,0.870918,0.873363,0.869855,0.876960,0.877108,0.889976,0.864608,0.877157
2,df_rdkit.csv,RandomUnderSampler,None,RandomForestClassifier,0.868091,0.871038,0.864294,0.878114,0.867938,0.880579,0.881437,0.888889,0.874109,0.880682
3,df_rdkit.csv,None,SelectKBest,RandomForestClassifier,0.858128,0.861716,0.852847,0.870985,0.857932,0.872135,0.872289,0.885086,0.859857,0.872331
4,df_rdkit.csv,SMOTENC,SelectKBest,RandomForestClassifier,0.862660,0.866091,0.857447,0.875161,0.862476,0.869723,0.870192,0.880779,0.859857,0.869880
5,df_rdkit.csv,RandomUnderSampler,SelectKBest,RandomForestClassifier,0.859642,0.862476,0.857973,0.867430,0.859531,0.868516,0.869148,0.878641,0.859857,0.868654
6,df_rdkit.csv,None,LDA,RandomForestClassifier,0.729260,0.738080,0.726801,0.750300,0.728917,0.780458,0.787879,0.773455,0.802850,0.780102
7,df_rdkit.csv,SMOTENC,LDA,RandomForestClassifier,0.735292,0.745039,0.729909,0.761589,0.734870,0.778046,0.779376,0.786925,0.771971,0.778143
8,df_rdkit.csv,RandomUnderSampler,LDA,RandomForestClassifier,0.723824,0.731044,0.724588,0.738404,0.723581,0.788902,0.791914,0.792857,0.790974,0.788869
9,df_rdkit.csv,None,FeatureAgglomeration,RandomForestClassifier,0.703290,0.707330,0.708665,0.706320,0.703255,0.692400,0.705202,0.686937,0.724466,0.691890


In [8]:
results_df.sort_values(by=['Val_Accuracy'], ascending=False, inplace = True)
results_df

,Fingerprint,Feature_Selection,Dim_Reduction,Classifier,CV_Mean_Accuracy,CV_Mean_F1,CV_Mean_Precision,CV_Mean_Recall,CV_Mean_ROC_AUC,Val_Accuracy,Val_F1,Val_Precision,Val_Recall,Val_ROC_AUC
17,df_circular.csv,RandomUnderSampler,None,RandomForestClassifier,0.883789,0.885733,0.884024,0.887643,0.883737,0.887817,0.888087,0.900000,0.876485,0.887997
16,df_circular.csv,SMOTENC,None,RandomForestClassifier,0.881374,0.883550,0.880269,0.887044,0.881293,0.886610,0.886747,0.899756,0.874109,0.886810
15,df_circular.csv,None,None,RandomForestClassifier,0.884997,0.887764,0.879452,0.896559,0.884826,0.884198,0.884615,0.895377,0.874109,0.884359
32,df_morgan.csv,RandomUnderSampler,None,RandomForestClassifier,0.881979,0.884255,0.880375,0.888238,0.881887,0.884198,0.885167,0.891566,0.878860,0.884283
30,df_morgan.csv,None,None,RandomForestClassifier,0.884997,0.887657,0.880233,0.895362,0.884838,0.881785,0.882775,0.889157,0.876485,0.881870
2,df_rdkit.csv,RandomUnderSampler,None,RandomForestClassifier,0.868091,0.871038,0.864294,0.878114,0.867938,0.880579,0.881437,0.888889,0.874109,0.880682
19,df_circular.csv,SMOTENC,SelectKBest,RandomForestClassifier,0.872624,0.874694,0.873632,0.876365,0.872583,0.879373,0.876847,0.910486,0.845606,0.879911
31,df_morgan.csv,SMOTENC,None,RandomForestClassifier,0.884394,0.886757,0.881828,0.891799,0.884281,0.878166,0.878460,0.890244,0.866983,0.878345
1,df_rdkit.csv,SMOTENC,None,RandomForestClassifier,0.869906,0.871928,0.870918,0.873363,0.869855,0.876960,0.877108,0.889976,0.864608,0.877157
18,df_circular.csv,None,SelectKBest,RandomForestClassifier,0.867493,0.869598,0.868821,0.871008,0.867455,0.875754,0.874543,0.897500,0.852732,0.876121
